In [14]:
import sys
import os

path = "../visualization"
sys.path.append(os.path.abspath(path))
from ase_nglview import view_ngl

import numpy as np
from ase.cluster.icosahedron import Icosahedron
from ase.build import bulk
from ase.io import write

def generate_spherical_cluster(symbol, structure, lattice_constant, target_N, cell_size, repeat=(6,6,6)):
    """
    bcc や fcc のバルク構造を十分大きく繰り返し、
    原点近傍から距離の近い順に target_N 個の原子を抜き出して球状クラスターを作成する。

    Parameters
    ----------
    symbol : str
        元素記号（例: 'Fe'）
    structure : str
        結晶構造 ('bcc', 'fcc' など)
    lattice_constant : float
        格子定数（Å）
    target_N : int
        取り出す原子数
    repeat : tuple of int
        バルクを繰り返す次数

    Returns
    -------
    atoms : ase.Atoms
        生成されたクラスター
    """
    # ① バルク構造を生成し、repeat で拡大
    bulk_system = bulk(symbol, structure, a=lattice_constant).repeat(repeat)

    # ② クラスターの中心をバルクのセル中心に設定
    cell = bulk_system.get_cell()
    center = np.sum(cell.array, axis=0) / 2.0

    # ③ すべての原子と中心との距離を計算 → 近い順に target_N 個を選択
    distances = np.linalg.norm(bulk_system.get_positions() - center, axis=1)
    indices = np.argsort(distances)[:target_N]

    # ④ 選択した原子のみ抜き出して返す
    cluster = bulk_system[indices]
    
    cluster.set_cell([cell_size]*3)
    cluster.center()
    cluster.set_pbc([True, True, True])
    return cluster

## Fe55 (icosahedral)

In [15]:
# Fe の格子定数（bcc, T=300K 近傍）約 2.866 Å
fe_a0 = 2.866

# 1) Fe55 はイコサヘドラル構造（noshells=3 → 1+12+42=55 atoms）
fe55 = Icosahedron('Fe', noshells=3, latticeconstant=fe_a0)
cell_size = 20.0  # Å

fe55.set_cell([cell_size]*3)
fe55.center()
fe55.set_pbc([True, True, True])

view_ngl(fe55)

NGLWidget()

In [16]:
write('Fe55.xyz', fe55)

## Fe80

In [21]:
fe80 = generate_spherical_cluster('Fe', 'bcc', fe_a0, target_N=80, cell_size=cell_size, repeat=(6,6,6))
view_ngl(fe80)

NGLWidget()

In [22]:
write('Fe80.xyz', fe80)

## Fe100

In [23]:
fe100 = generate_spherical_cluster('Fe', 'bcc', fe_a0, target_N=100, cell_size=cell_size, repeat=(6,6,6))
view_ngl(fe100)

NGLWidget()

In [24]:
write('Fe100.xyz', fe100)

## Fe120

In [25]:
fe120 = generate_spherical_cluster('Fe', 'bcc', fe_a0, target_N=120, cell_size=cell_size, repeat=(6,6,6))
view_ngl(fe120)

NGLWidget()

In [26]:
write('Fe120.xyz', fe120)